In [ ]:
#In this example, we will get course suggestions to improve our career 
# based on our CV uploaded IN PDF format
#05/10/2025

In [ ]:
pip install python-dotenv openai PyPDF2 ipywidgets

In [ ]:
#imports

import io
import os
import PyPDF2
import ipywidgets as widgets
from IPython.display import display, clear_output
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
# Load environment variables
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Validate API key
if not api_key:
    print("❌ No API key was found - please head over to the troubleshooting notebook to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("⚠️ An API key was found, but it doesn't start with sk-proj-; please check you're using the right key.")
elif api_key.strip() != api_key:
    print("⚠️ An API key was found, but it looks like it might have extra spaces or tabs. Please fix.")
else:
    print("✅ API key found and looks good!")


In [ ]:
# Create OpenAI client
openai = OpenAI(api_key=api_key)

In [ ]:
# File upload widget
upload_widget = widgets.FileUpload(
    accept='.pdf',
    multiple=False
)
display(upload_widget)

In [ ]:
# Extract text from PDF
def extract_text(uploaded_file):
    uploaded_info = list(uploaded_file.values())[0] if isinstance(uploaded_file, dict) else uploaded_file[0]
    file_content = uploaded_info['content']
    file_stream = io.BytesIO(file_content)
    reader = PyPDF2.PdfReader(file_stream)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text


In [ ]:
# Get course suggestions using OpenAI API
def get_course_suggestions(resume_text):
    system_prompt = "You are a career coach that suggests valuable courses based on a resume."
    user_prompt = f"Here is my resume:\n\n{resume_text}\n\nWhat courses could help me improve this resume for better job opportunities?"

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )
    return response.choices[0].message.content


In [ ]:
# Analyze button callback
def on_analyze_click(b):
    with output:
        clear_output()
        if not upload_widget.value:
            print("❌ Please upload a PDF resume first.")
            return

        try:
            resume_text = extract_text(upload_widget.value)
            print("✅ Resume uploaded. Getting course suggestions...\n")
            suggestions = get_course_suggestions(resume_text)
            print("💡 Suggested Courses:")
            print(suggestions)
        except Exception as e:
            print(f"❌ An error occurred: {e}")


In [ ]:
# Analyze button
analyze_button = widgets.Button(description="Analyze Resume", button_style='primary')
analyze_button.on_click(on_analyze_click)
display(analyze_button)

# Output area
output = widgets.Output()
display(output)